In [1]:
#cloning dataset from github
# ! git clone https://github.com/nikhil-xyz/Text-classification.git

In [2]:
import pandas as pd
data = pd.read_csv('Text-classification/go_emotions_dataset.csv', on_bad_lines='skip')

In [3]:
data = data.head(10000)

In [4]:
data = pd.get_dummies(data, columns = ['example_very_unclear'])
data.drop(['example_very_unclear_False'], axis=1, inplace=True)
data = data.rename(columns = {'example_very_unclear_False':'example_very_unclear'})
data.shape

(10000, 31)

In [5]:
data.isnull().sum()

id                           0
text                         0
admiration                   0
amusement                    0
anger                        0
annoyance                    0
approval                     0
caring                       0
confusion                    0
curiosity                    0
desire                       0
disappointment               0
disapproval                  0
disgust                      0
embarrassment                0
excitement                   0
fear                         0
gratitude                    0
grief                        0
joy                          0
love                         0
nervousness                  0
optimism                     0
pride                        0
realization                  0
relief                       0
remorse                      0
sadness                      0
surprise                     0
neutral                      0
example_very_unclear_True    0
dtype: int64

In [6]:
# removing panctuations
import string
data['text'] = data['text'].apply(lambda word:[letters.lower() for letters in word if letters not in string.punctuation])
data['text'] = data['text'].apply(lambda word:''.join(word))

In [7]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tarun\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Tarun\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Tarun\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [8]:
wnl = WordNetLemmatizer()

In [9]:
def preprocess(text):
  text = text.lower()
  text = text.split()
  text = [wnl.lemmatize(word) for word in text if not word in stopwords.words('english')]
  text = ' '.join(text)
  return text

In [10]:
data['text'] = data['text'].apply(preprocess)

In [11]:
data.iloc[1]['text']

'sexuality shouldn’t grouping category make different othet ppl imo fit definition grouping'

In [12]:
X = list(data['text'])
y = list(data[data.columns[2:]].values)

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [14]:
y_test[7]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0], dtype=int64)

In [15]:
!pip install transformers

In [16]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [17]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [30]:
!pip install torch

ERROR: Could not find a version that satisfies the requirement torch (from versions: none)
ERROR: No matching distribution found for torch


In [29]:
import torch
import numpy as np
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.FloatTensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

ModuleNotFoundError: No module named 'torch'

In [ ]:
train_dataset = Dataset(train_encodings, y_train)
test_dataset = Dataset(test_encodings, y_test)

In [ ]:
sample = train_dataset[0]
sample.keys()

In [ ]:
sample['labels']

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=5,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100
)


In [ ]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", problem_type="multi_label_classification", return_dict=True, num_labels=29)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

In [ ]:
trainer.train()

In [ ]:
output = trainer.predict(test_dataset)[1]

In [ ]:
!pip install scikit-learn

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
accuracy = accuracy_score(y_test, output)
# precision = precision_score(y_test, output)
# recall = recall_score(y_test, output)
# f1 = f1_score(y_test, output)

print("accuracy: ", accuracy)
# print("precision: ", precision)
# print("Recall: ", recall)
# print("F1-score: ", f1)

In [ ]:
trainer.save_model('assignment')